In [1]:
!pip install python-dotenv langchain-tavily langchain-community langchain-core

  Using cached langchain_tavily-0.2.11-py3-none-any.whl.metadata (22 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
#!/usr/bin/env python3
"""
Simple Jupyter Web Browsing Agent
Direct print output - no display issues!

Usage:
    agent = WebBrowsingAgent()
    agent.query("What is the capital of France?")
"""

import os
import json
import http.client
import requests
from typing import Dict, Optional
from dataclasses import dataclass
from enum import Enum
import re
from dotenv import load_dotenv

# LangChain imports
from langchain.tools import tool

# Load environment variables
load_dotenv()

class ConfidenceLevel(Enum):
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"

@dataclass
class QueryAnalysis:
    confidence: ConfidenceLevel
    reasoning: str
    requires_web_search: bool
    search_query: Optional[str] = None

In [4]:
# ===========================================
# @TOOL FUNCTIONS
# ===========================================

@tool
def search_web_serper(query: str) -> str:
    """Search the web using Serper API for current information and news."""
    api_key = os.getenv("SERPER_API_KEY")
    if not api_key:
        return "⚠️ SERPER_API_KEY not configured. Please add it to your .env file to enable web search."

    try:
        conn = http.client.HTTPSConnection("google.serper.dev")
        payload = json.dumps({
            "q": query,
            "gl": "us",
            "num": 3
        })
        headers = {
            'X-API-KEY': api_key,
            'Content-Type': 'application/json'
        }

        conn.request("POST", "/search", payload, headers)
        res = conn.getresponse()
        data = res.read()
        conn.close()

        results = json.loads(data.decode("utf-8"))

        if "organic" not in results or not results["organic"]:
            return "No search results found for your query."

        # Format results
        formatted = "🔍 Web Search Results:\n\n"
        for i, result in enumerate(results["organic"][:3], 1):
            title = result.get("title", "")
            snippet = result.get("snippet", "")
            link = result.get("link", "")

            if title and snippet:
                formatted += f"{i}. {title}\n"
                formatted += f"   {snippet}\n"
                if link:
                    formatted += f"   Source: {link}\n"
                formatted += "\n"

        return formatted.strip()

    except Exception as e:
        return f"Search error: {str(e)[:100]}"

@tool
def get_current_bitcoin_price() -> str:
    """Get the current Bitcoin price in USD from CoinDesk API."""
    try:
        response = requests.get("https://api.coindesk.com/v1/bpi/currentprice/USD.json", timeout=10)
        data = response.json()

        price = data['bpi']['USD']['rate']
        last_updated = data['time']['updated']

        return f"₿ Current Bitcoin Price: ${price}\nLast Updated: {last_updated}\nSource: CoinDesk Bitcoin Price Index"

    except Exception as e:
        return f"Unable to fetch Bitcoin price: {str(e)[:50]}"

@tool
def get_knowledge_base_answer(topic: str) -> str:
    """Provide answers for well-established factual topics."""
    topic_lower = topic.lower()

    # Knowledge base with common topics
    knowledge_base = {
        "france": "The capital of France is Paris. Paris is located in north-central France on the Seine River and serves as the country's political, economic, and cultural center. It has a population of over 2 million within the city limits and over 12 million in the metropolitan area.",

        "compound_interest": "Compound interest is interest calculated on the initial principal and all previously earned interest.\n\nFormula: A = P(1 + r/n)^(nt)\n- A = Final amount\n- P = Principal (initial amount)\n- r = Annual interest rate (decimal)\n- n = Compounding frequency per year\n- t = Time in years\n\nExample: $1,000 at 5% compounded annually for 10 years becomes $1,628.89.",

        "machine_learning": "Machine learning is a branch of artificial intelligence that enables computers to learn and make decisions from data without being explicitly programmed for every task.\n\nTypes:\n- Supervised Learning: Learning with labeled examples\n- Unsupervised Learning: Finding patterns in unlabeled data\n- Reinforcement Learning: Learning through interaction and feedback\n\nApplications: Image recognition, natural language processing, recommendation systems, autonomous vehicles.",

        "photosynthesis": "Photosynthesis is the process by which plants convert sunlight, carbon dioxide, and water into glucose and oxygen.\n\nEquation: 6CO₂ + 6H₂O + light energy → C₆H₁₂O₆ + 6O₂\n\nThis process occurs in chloroplasts and is essential for life on Earth as it produces oxygen and serves as the foundation of most food chains.",
    }

    # Search for matching knowledge
    for key, value in knowledge_base.items():
        if key in topic_lower:
            return value

    # Check for capital cities specifically
    if "capital" in topic_lower:
        capitals = {
            "france": "Paris",
            "germany": "Berlin",
            "italy": "Rome",
            "spain": "Madrid",
            "uk": "London",
            "united kingdom": "London",
            "usa": "Washington, D.C.",
            "united states": "Washington, D.C.",
            "canada": "Ottawa",
            "australia": "Canberra",
            "japan": "Tokyo"
        }

        for country, capital in capitals.items():
            if country in topic_lower:
                return f"The capital of {country.title()} is {capital}."

    # Generic response for unknown topics
    return f"This appears to be a factual question about '{topic}'. For the most current and comprehensive information, I recommend using web search."

In [5]:
# ===========================================
# MAIN AGENT CLASS
# ===========================================

class WebBrowsingAgent:
    """Simple web browsing agent with direct print output."""

    def __init__(self):
        self.tools = {
            'search_web_serper': search_web_serper,
            'get_current_bitcoin_price': get_current_bitcoin_price,
            'get_knowledge_base_answer': get_knowledge_base_answer
        }

        self.current_info_keywords = [
            'current', 'latest', 'recent', 'now', 'today', 'price', 'news'
        ]

        self.dynamic_topics = [
            'bitcoin', 'cryptocurrency', 'weather', 'news', 'stock'
        ]

        self.factual_indicators = [
            'capital of', 'formula for', 'definition of', 'how to calculate',
            'what is', 'explain'
        ]

        print("🤖 Web Browsing Agent initialized!")
        print("Available tools:", list(self.tools.keys()))

    def analyze_query_confidence(self, query: str) -> QueryAnalysis:
        """Analyze query confidence."""
        query_lower = query.lower().strip()

        # Check for current information needs
        needs_current_info = any(kw in query_lower for kw in self.current_info_keywords)
        has_dynamic_topics = any(topic in query_lower for topic in self.dynamic_topics)
        has_factual_indicators = any(ind in query_lower for ind in self.factual_indicators)

        # Decision logic
        if 'bitcoin' in query_lower and 'price' in query_lower:
            confidence = ConfidenceLevel.LOW
            reasoning = "Bitcoin price query requires real-time data"
            requires_search = True
        elif needs_current_info or has_dynamic_topics:
            confidence = ConfidenceLevel.LOW
            reasoning = "Query requires current information"
            requires_search = True
        elif has_factual_indicators or 'capital' in query_lower:
            confidence = ConfidenceLevel.HIGH
            reasoning = "Query involves well-established facts"
            requires_search = False
        elif 'who is' in query_lower:
            confidence = ConfidenceLevel.MEDIUM
            reasoning = "Query about specific person"
            requires_search = True
        else:
            confidence = ConfidenceLevel.MEDIUM
            reasoning = "Uncertain about information completeness"
            requires_search = True

        # Generate search query
        search_query = self._optimize_search_query(query) if requires_search else None

        return QueryAnalysis(
            confidence=confidence,
            reasoning=reasoning,
            requires_web_search=requires_search,
            search_query=search_query
        )

    def _optimize_search_query(self, query: str) -> str:
        """Optimize search query."""
        stop_words = {'what', 'who', 'is', 'the', 'a'}
        words = query.lower().replace('?', '').split()
        filtered_words = [word for word in words if word not in stop_words]
        return ' '.join(filtered_words[:6])

    def select_tool(self, query: str, analysis: QueryAnalysis) -> str:
        """Select appropriate tool."""
        query_lower = query.lower()

        if not analysis.requires_web_search:
            return 'get_knowledge_base_answer'
        elif 'bitcoin' in query_lower and 'price' in query_lower:
            return 'get_current_bitcoin_price'
        else:
            return 'search_web_serper'

    def query(self, user_query: str, verbose: bool = False):
        """
        Process a query and print results directly.

        Args:
            user_query: The user's question
            verbose: Show detailed analysis
        """
        print(f"\n{'='*60}")
        print(f"🔍 QUERY: {user_query}")
        print('='*60)

        if not user_query or not user_query.strip():
            print("❌ Please provide a valid query.")
            return

        try:
            # Analyze the query
            analysis = self.analyze_query_confidence(user_query)
            selected_tool_name = self.select_tool(user_query, analysis)

            # Show analysis
            confidence_emoji = {"high": "🟢", "medium": "🟡", "low": "🔴"}
            emoji = confidence_emoji.get(analysis.confidence.value, "⚪")

            print(f"{emoji} CONFIDENCE: {analysis.confidence.value.upper()}")
            print(f"💭 REASONING: {analysis.reasoning}")
            print(f"🛠️  TOOL SELECTED: {selected_tool_name}")
            print(f"🔍 WEB SEARCH: {'Yes' if analysis.requires_web_search else 'No'}")

            if verbose and analysis.search_query:
                print(f"🔎 SEARCH QUERY: {analysis.search_query}")

            print(f"\n📋 ANSWER:")
            print("-" * 40)

            # Execute the tool
            tool = self.tools[selected_tool_name]

            if selected_tool_name == 'get_knowledge_base_answer':
                answer = tool.invoke({"topic": user_query})
            elif selected_tool_name == 'get_current_bitcoin_price':
                answer = tool.invoke({})
            else:
                search_query = analysis.search_query or user_query
                answer = tool.invoke({"query": search_query})

            # Print the answer
            print(answer)
            print("-" * 40)
            print("✅ Query completed!")

        except Exception as e:
            print(f"❌ Error: {str(e)}")

    def interactive_mode(self):
        """Interactive mode with direct input/output."""
        print("\n🤖 Interactive Mode Started")
        print("Commands: 'help', 'quit', 'verbose on/off'")
        print("="*50)

        verbose = False

        while True:
            try:
                user_input = input("\n🔍 Enter query: ").strip()

                if not user_input:
                    continue

                if user_input.lower() == 'quit':
                    print("👋 Goodbye!")
                    break
                elif user_input.lower() == 'help':
                    print("\nCommands:")
                    print("- help: Show this help")
                    print("- quit: Exit")
                    print("- verbose on/off: Toggle detailed analysis")
                    print("\nExample queries:")
                    print("- What is the capital of France?")
                    print("- Current Bitcoin price")
                    print("- Who is Elon Musk?")
                    continue
                elif user_input.lower().startswith('verbose'):
                    if 'on' in user_input.lower():
                        verbose = True
                        print("🔧 Verbose mode: ON")
                    elif 'off' in user_input.lower():
                        verbose = False
                        print("🔧 Verbose mode: OFF")
                    else:
                        verbose = not verbose
                        print(f"🔧 Verbose mode: {'ON' if verbose else 'OFF'}")
                    continue

                # Process query
                self.query(user_input, verbose=verbose)

            except KeyboardInterrupt:
                print("\n👋 Session ended!")
                break
            except Exception as e:
                print(f"❌ Error: {e}")

In [31]:
# ===========================================
# QUICK TEST FUNCTION
# ===========================================

def test_agent():
    """Quick test of the agent."""
    agent = WebBrowsingAgent()

    test_queries = [
        "What is the capital of France?",
        "Current Bitcoin price",
        "Who is Ojasw Kant?"
    ]

    for query in test_queries:
        agent.query(query)

# Usage instructions
print("""
🚀 USAGE INSTRUCTIONS:

# Initialize agent
agent = WebBrowsingAgent()

# Single query
agent.query("What is the capital of France?")

# With verbose analysis
agent.query("Current Bitcoin price", verbose=True)

# Interactive mode
# agent.interactive_mode()

# Quick test
# test_agent()
""")


🚀 USAGE INSTRUCTIONS:

# Initialize agent
agent = WebBrowsingAgent()

# Single query
agent.query("What is the capital of France?")

# With verbose analysis
agent.query("Current Bitcoin price", verbose=True)

# Interactive mode
# agent.interactive_mode()

# Quick test
# test_agent()



In [33]:
agent = WebBrowsingAgent()

🤖 Web Browsing Agent initialized!
Available tools: ['search_web_serper', 'get_current_bitcoin_price', 'get_knowledge_base_answer']


In [37]:
agent.query("Find out how many car accidents happened last year?")


🔍 QUERY: Find out how many car accidents happened last year?
🟡 CONFIDENCE: MEDIUM
💭 REASONING: Uncertain about information completeness
🛠️  TOOL SELECTED: search_web_serper
🔍 WEB SEARCH: Yes

📋 ANSWER:
----------------------------------------
🔍 Web Search Results:

1. Data | NHTSA
   ... Reports which provide information on crashes at the national and State levels. NCSA also produces many reports on Seat Belt Use in the States and Territories.
   Source: https://www.nhtsa.gov/data

2. Crash Reports - Ohio Department of Public Safety
   Search for crash reports or explore traffic crash data. Search for a Crash Report. Search unofficial crash reports as reported to ODPS for past five years plus ...
   Source: https://publicsafety.ohio.gov/wps/portal/gov/odps/what-we-do/crash-reports/

3. CrashStats - NHTSA - DOT
   Welcome to NHTSA's National Center for Statistics and Analysis (NCSA) Motor Vehicle Traffic Crash Data Resource Page
   Source: https://crashstats.nhtsa.dot.gov/
------------